In [19]:
import os
import cv2
import mediapipe as mp
import numpy as np
import math
import time

In [20]:
folder="AAROHAN"
if not os.path.exists(folder):
    os.makedirs(folder)
    print(f"Created directory: {folder}")
counter=0

In [21]:
cap=cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
imgSize=224

In [22]:
mpHands=mp.solutions.hands
hands=mpHands.Hands()
mpDraw=mp.solutions.drawing_utils

In [23]:
pTime=0
cTime=0

In [24]:
def fps_calculation(img,pTime):
    cTime=time.time()
    fps=int(1/(cTime-pTime))
    pTime=cTime
    cv2.putText(
        img, f"FPS:{fps}", (10, 70), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2
    )
    return pTime

In [25]:
def crop_hands(img, handLms):
    h, w, c = img.shape
    x_min, y_min = w, h
    x_max, y_max = 0, 0

    for lm in handLms.landmark:
        x, y = int(lm.x * w), int(lm.y * h)

        x_min = min(x_min, x)
        y_min = min(y_min, y)

        x_max = max(x_max, x)
        y_max = max(y_max, y)

    padding = 40
    x_min = max(0, x_min - padding)
    y_min = max(0, y_min - padding)
    x_max = min(w, x_max + padding)
    y_max = min(h, y_max + padding)

    cropped_img = img[y_min:y_max, x_min:x_max]

    return cropped_img, (x_min, y_min, x_max, y_max)

In [26]:
while True:
    success, img = cap.read()
    imgRgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRgb)

    imgLandmarksOnly = np.ones((720, 1280, 3), np.uint8) * 255
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            cropped_image, bbox = crop_hands(imgLandmarksOnly, handLms)
            
            

            if cropped_image.size == 0:
                print("Warning: Empty cropped image. Skipping frame.")
                continue

            x_min, y_min, x_max, y_max = bbox
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
            mpDraw.draw_landmarks(imgLandmarksOnly, handLms, mpHands.HAND_CONNECTIONS)

            h, w = cropped_image.shape[:2]

            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

            aspect_ratio = h / w

            if aspect_ratio > 1:
                k = imgSize / h
                wCal = int(math.ceil(k * w))
                imgResize = cv2.resize(cropped_image, (wCal, imgSize))
                imgResizeShape = imgResize.shape
                wGap = math.ceil((imgSize - wCal) //2)
                imgWhite[ : , wGap : wCal + wGap] = imgResize

            else:
                k = imgSize / w
                hCal = int(math.ceil(k * h))
                imgResize = cv2.resize(cropped_image, (imgSize, hCal))
                imgResizeShape = imgResize.shape
                hGap = math.ceil((imgSize - hCal) // 2)
                imgWhite[hGap : hGap + hCal,  :] = imgResize
            

            cv2.imshow("Hand Landmarks Only", imgLandmarksOnly)
            cv2.imshow("ImageWhite", imgWhite)
            cv2.imshow("Cropped Hand", cropped_image)

    fps = fps_calculation(img, pTime)
    cv2.imshow("Image", img)

    key1 = cv2.waitKey(1) & 0xFF
    if key1 == ord("s"):
        counter += 1
        cv2.imwrite(f"{folder}/Image{time.time()}.jpg", imgWhite)
        print(counter)

    key2=cv2.waitKey(1)
    if key2 == ord("q") or key2 == 27:
        break


In [27]:
cap.release()
cv2.destroyAllWindows()